In [5]:
# !pip install websocket
# !pip install websocket_client
# !pip install prettytable

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [1]:
import pandas as pd

In [2]:
from csmarapi.ReportUtil import ReportUtil
from csmarapi.CsmarService import CsmarService
csmar = CsmarService()
# login(account,pwd,lang)
csmar.login('15259506516', 'oncezzh757699')

Fri 13 Sep 2024 08:46:14 CsmarService.py INFO User login succeed


In [3]:
tables = csmar.getListTables('机构投资者')
ReportUtil(tables)

Fri 13 Sep 2024 08:46:36 connectionpool.py WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001FE73B79700>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。')': /api/csmar-main/python/listTables?dbName=%E6%9C%BA%E6%9E%84%E6%8A%95%E8%B5%84%E8%80%85
Fri 13 Sep 2024 08:46:57 connectionpool.py WARNING Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001FE73B795E0>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。')': /api/csmar-main/python/listTables?dbName=%E6%9C%BA%E6%9E%84%E6%8A%95%E8%B5%84%E8%80%85


+-----------------------+--------------------+------------+------------+
|         table         |     tableName      | startTime  |  endTime   |
+-----------------------+--------------------+------------+------------+
| INI_HolderSystematics | 机构持股分类统计表 | 1998-06-30 | 2024-06-30 |
| INI_Holder_IncrOrDesr |  机构增减持统计表  | 1998-06-30 | 2024-06-30 |
|   INI_Holder_Detail   |   机构持股明细表   | 1998-06-30 | 2024-06-30 |
+-----------------------+--------------------+------------+------------+


In [4]:
fields = csmar.getListFields('INI_Holder_Detail')
ReportUtil(fields)

+---------------------+---------------------------+-----------+----------+----------+
|        field        |         fieldName         | fieldType | ableNull | fieldKey |
+---------------------+---------------------------+-----------+----------+----------+
|        Symbol       |          股票代码         |  varchar  |   YES    |   Code   |
|       EndDate       |          截止日期         |  varchar  |   YES    |   Date   |
|    SystematicsID    |        机构分类编码       |  varchar  |   YES    |          |
|     Systematics     |          机构分类         |  varchar  |   YES    |          |
|   ShareHolderName   |          持股机构         |  varchar  |   YES    |          |
|     CategoryCode    |        股东类别编码       |  varchar  |   YES    |          |
|      Holdshares     |          持股数量         |  decimal  |   YES    |          |
|    HoldProportion   |    持股比例(相对总股本)   |  decimal  |   YES    |          |
|        Price        |          股票价格         |  decimal  |   YES    |          |
|     LastEnd

In [5]:
columns = [field['field'] for field in fields]
columns

['Symbol',
 'EndDate',
 'SystematicsID',
 'Systematics',
 'ShareHolderName',
 'CategoryCode',
 'Holdshares',
 'HoldProportion',
 'Price',
 'LastEndDate',
 'LastShareHolderID',
 'LastHoldshares',
 'LastHoldProportion',
 'LastPrice',
 'RightsRatio',
 'Status',
 'IndustryCode',
 'IndustryName',
 'Source',
 'ShareHolderID',
 'InstitutionID',
 'FundID',
 'HoldProportion1',
 'LastHoldProportion1',
 'IndustryCode1',
 'IndustryName1']

In [6]:
# 分批下载并打包数据
batch_size = 200000
offset = 0

# 定义开始和结束日期，20年分多个时间段进行查询
time_ranges = [
    # ('2004-01-01', '2009-12-31'),  # 第一个6年
    # ('2010-01-01', '2015-12-31'),  # 第二个6年
    # ('2016-01-01', '2021-12-31'),  # 第三个6年
    ('2022-01-01', '2023-12-31')   # 最后两年
]

# 遍历每个时间区间
for start_date, end_date in time_ranges:
    offset = 0  # 每个时间段从偏移量0开始

    while True:
        # 构建分页查询条件
        condition = f"limit {offset},{batch_size}"
        
        # 查询数据
        data_part = csmar.query_df(columns, condition, 
                                   'INI_Holder_Detail', 
                                   start_date, end_date)
        
        # 调用打包下载函数
        csmar.getPackResultExt(columns, condition, 
                               'INI_Holder_Detail', 
                               start_date, end_date)
        
        # 检查返回的数据量是否小于分页大小，停止查询
        if len(data_part) < batch_size:
            break
        
        # 更新偏移量
        offset += batch_size


Fri 13 Sep 2024 08:47:28 CsmarService.py ERROR Please don't submit your request repeatedly.【请不要重复提交相同请求】
Fri 13 Sep 2024 08:47:29 CsmarService.py INFO packaging return code：signCode=1284072942737592320
Fri 13 Sep 2024 08:47:30 CsmarService.py INFO downloading...


|▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇|100%

Fri 13 Sep 2024 08:51:09 CsmarService.py INFO Package successfully. File size is：90.3MB


Fri 13 Sep 2024 08:51:15 CsmarService.py INFO Generate the local file path as ：c:\csmardata\zip\1284072942737592320.zip


TypeError: object of type 'NoneType' has no len()